### Instalações

In [2]:
import pandas as pd

from sklearn.preprocessing import LabelEncoder

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(0)
plt.style.use("ggplot")

import tensorflow
from tensorflow.keras.preprocessing.sequence import pad_sequences
from gensim.models import Word2Vec
from keras.models import Sequential
from keras import Model, Input
from tensorflow.keras.layers import LSTM, Dense, BatchNormalization, Dropout, Conv1D, MaxPooling1D, Embedding, TimeDistributed, Bidirectional,GlobalMaxPooling1D

#!pip install -U gensim
from gensim.models import Word2Vec

#!pip install livelossplot
from livelossplot.tf_keras import PlotLossesCallback

#!pip install seqeval
from seqeval.metrics import f1_score, classification_report, precision_score, recall_score, accuracy_score
from seqeval.scheme import IOB2
#! pip install plot_keras_history
from plot_keras_history import plot_history
from keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

[nltk_data] Downloading package punkt to /Users/lucelia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2023-07-11 23:01:33.086438: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Dados

In [3]:
import pandas as pd

df = pd.read_csv('./CSVs/V2/DODFCorpus_contratos_licitacoes_v2.csv', dtype=str)
df = df.drop(['Unnamed: 0','Unnamed: 0.1'], axis =1)

In [4]:
##Insere um espaço entre as entidade e insere um espaco apos :  : -> Processo:0...
import re
def correct_space_before_numeric_entities(string):
    result = re.sub(r'([A-Za-z]:)[0-9]', r'\1 ', string)
    result = result.replace("\n", " ")
    return result

In [5]:
df['texto']= df['texto'].map(correct_space_before_numeric_entities)

In [6]:
df.tipo_rel.unique()

array(['REL_AVISO_LICITACAO', 'REL_SUSPENSAO_LICITACAO',
       'REL_EXTRATO_CONTRATO', 'REL_ADITAMENTO_CONTRATO',
       'REL_ANUL_REVOG_LICITACAO', 'REL_EXTRATO_CONVENIO'], dtype=object)

In [7]:
#Concatena o id_ato com o id_dodf
df['id_ato'] = df['id_dodf'] + '-' + df['id_rel']
#data

In [8]:
#Lista o tipo de relatório
result =df.groupby('tipo_rel')['id_ato'].nunique()
print(result.sort_values(ascending=False))

tipo_rel
REL_EXTRATO_CONTRATO        1734
REL_ADITAMENTO_CONTRATO     1551
REL_AVISO_LICITACAO          639
REL_SUSPENSAO_LICITACAO       82
REL_ANUL_REVOG_LICITACAO      52
REL_EXTRATO_CONVENIO          32
Name: id_ato, dtype: int64


In [9]:
tipo_ato = df.query("tipo_rel == 'REL_EXTRATO_CONTRATO'").reset_index(drop=True)

In [10]:
#Lista as entidades do tipo de ato
result = tipo_ato.groupby('tipo_ent')['id_ato'].nunique()
print(result)
#print(result.sort_values(ascending=False))

tipo_ent
EXTRATO_CONTRATO            1734
cnpj_entidade_contratada     479
cnpj_orgao_contratante       135
codigo_siggo                 175
data_assinatura_contrato    1286
entidade_contratada         1721
fonte_recurso               1317
natureza_despesa            1068
nome_responsavel             185
nota_empenho                1214
numero_contrato             1714
objeto_contrato             1724
orgao_contratante           1700
processo_gdf                1721
programa_trabalho           1280
unidade_orcamentaria        1045
valor_contrato              1566
vigencia_contrato           1664
Name: id_ato, dtype: int64


# Geração do IOB

In [11]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [12]:
#Carrega o modelo do spacy em portugues
from mimetypes import init
import re
import spacy
from spacy.lang.pt.examples import sentences 
from spacy.tokenizer import Tokenizer
from spacy.util import compile_prefix_regex, compile_infix_regex, compile_suffix_regex
from spacy.tokens import DocBin

class spacy_tokenizer():
    def __init__(self):
        #self.nlp = spacy.load('pt_core_news_sm', disable=["ner", "lemmatizer"])
        self.nlp = spacy.load('pt_core_news_sm', disable = ['parser','ner'])
    
    def tokenize(self, texto):
            
        doc = self.nlp(texto)
        return[t.text.strip() for t in doc if t.text.strip()]
        

/Users/lucelia/Documents/GitHub/experiments/members/lucelia/venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
class iob_transformer():
    
    def __init__(self, coluna_id_ato: str, coluna_texto_entidade: str,
                 coluna_tipo_entidade: str, keep_punctuation: bool = False,
                 return_df: bool = False, tokenizer_tipo: str = "Função"):
        self.coluna_id_ato = coluna_id_ato
        self.coluna_texto_entidade = coluna_texto_entidade
        self.coluna_tipo_entidade = coluna_tipo_entidade
        self.tokenizer_tipo = tokenizer_tipo
        if not keep_punctuation: #False
            #self.tokenizer = RegexpTokenizer('\w+')
            self.tokenizer = spacy_tokenizer()
        else:
            self.tokenizer = False
        self.return_df = return_df

    
    def fit(self, X, y=None, **fit_params):
        return self
    
    def gera_listas_atos_iobs(self, df):
        
        def _inclui_tags_vazias(texto_iob):
            texto_ato_iob = texto_iob.copy()
            for idx, token in enumerate(texto_ato_iob):
                if token[0:2] == 'B-':# in token:
                    pass
                elif token[0:2] == 'I-':# in token:
                    pass
                else:
                    texto_ato_iob[idx] = 'O'
            
            return texto_ato_iob
        
        def _constroi_iob(texto_ent, tipo_ent):
            if self.tokenizer:
                texto_ent_tok = self.tokenizer.tokenize(texto_ent)
                
                
            else:
                texto_ent_tok = word_tokenize(texto_ent)
            iob_entidade = []
            for index_token, token in enumerate(texto_ent_tok):
                # primeiro token?
                if index_token == 0:
                    palavra = 'B-'+ tipo_ent
                    iob_entidade.append(palavra)
                # é o segundo token?
                else:
                    palavra = 'I-'+ tipo_ent
                    iob_entidade.append(palavra)
            # salva tupla contendo texto tokenizado e iob correspondente
            if self.tokenizer:
                tup_entidade = (self.tokenizer.tokenize(texto_ent), iob_entidade)
            
            else:
                tup_entidade = (word_tokenize(texto_ent), iob_entidade)
            return tup_entidade
        
        def _match_iob_texto_ato(texto_entidade_tok, iob_ato):
            texto_ato_iob = texto_entidade_tok.copy()
            #print(iob_ato)
            for tupla in iob_ato:
                 # checa se o texto de referência existe
                if tupla[0]:
                    for i in range(len(texto_entidade_tok)):
                        # checa se a tag existe
                        if tupla[0][0]:
                            # match primeiro token
                            if texto_entidade_tok[i] == tupla[0][0]:
                                # a sequência de tokens de texto_entidade_token na
                                # posição encontrada é igual aos tokens da entidade?
                                if texto_entidade_tok[i:i+len(tupla[0])] == tupla[0]:
                                    texto_ato_iob[i:i+len(tupla[0])] = tupla[1]
            
            return texto_ato_iob

        atos = []
        lista_labels = []
        id_atos = set()
        for row in df.iterrows():
            id_ato = df.iloc[row[0]][self.coluna_id_ato]
            texto_ato = []
            texto_ato_iob = []
            if id_ato not in id_atos:
                id_atos.add(id_ato)
                lista_ids = list(df.query(f'{self.coluna_id_ato} == "{id_ato}"').index)
                # print(lista_ids)
                iob_ato = []
                # todas as anotações que não são o ato inteiro
                for index in lista_ids:
                    texto_entidade = df.iloc[index][self.coluna_texto_entidade]
                    tipo_entidade = df.iloc[index][self.coluna_tipo_entidade]
                    #Lucelia if isinstance(self.tokenizer, RegexpTokenizer):
                    if self.tokenizer_tipo == "Função":
                        texto_entidade_tok = self.tokenizer.tokenize(texto_entidade)

                    else:
                        texto_entidade_tok = word_tokenize(texto_entidade)
                    if not tipo_entidade.isupper():
                        tup_entidade = _constroi_iob(texto_entidade, tipo_entidade)
                        iob_ato.append(tup_entidade)
                # anotação do ato inteiro
                for index in lista_ids:
                    texto_entidade = df.iloc[index][self.coluna_texto_entidade]
                    tipo_entidade = df.iloc[index][self.coluna_tipo_entidade]
                    #Lucelia if self.tokenizer:
                    if self.tokenizer_tipo == "Função":
                        texto_entidade_tok = self.tokenizer.tokenize(texto_entidade)
                    else:
                        texto_entidade_tok = word_tokenize(texto_entidade)
                    if tipo_entidade.isupper():
                        texto_ato = texto_entidade_tok
                        texto_ato_iob = _match_iob_texto_ato(texto_entidade_tok, iob_ato)
                texto_ato_iob = _inclui_tags_vazias(texto_ato_iob)
                atos.append(texto_ato)
                lista_labels.append(texto_ato_iob)
        
        return atos, lista_labels

    def create_iob_df(self, atos, lista_labels):
        rows_list = []
        dict1 = {
                'Sentence_idx': -1,
                'Word': 'UNK',
                'Tag': 'O'
            }
        rows_list.append(dict1)
        id_ato = 0
        for ato, labels in zip(atos, lista_labels):
            for word, label in zip(ato, labels):
                dict1 = {
                    'Sentence_idx': id_ato,
                    'Word': word,
                    'Tag': label
                }
                rows_list.append(dict1)
                #print(word, label)
            id_ato += 1
        new_df = pd.DataFrame(rows_list)

        return new_df    
    
    def transform(self, df, **transform_params):
        dataframe = df.copy()
        dataframe = dataframe.reset_index(drop=True)
        atos, lista_labels = self.gera_listas_atos_iobs(dataframe)
        if self.return_df:
            iob_df = self.create_iob_df(atos, lista_labels)
            return iob_df
        else:
            return atos, lista_labels

In [15]:
iob = iob_transformer('id_ato','texto','tipo_ent', keep_punctuation=False, return_df=False)

/Users/lucelia/Documents/GitHub/experiments/members/lucelia/venv/lib/python3.9/site-packages/spacy/language.py:1895: UserWarning: [W123] Argument disable with value ['parser', 'ner'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(


In [16]:
acts, labels = iob.transform(tipo_ato)

In [76]:
#acts

In [17]:
act_name = 'EXTRATO_CONVENIO' 

In [18]:
max_length = 400

### Ajustes das labels e dicionários

In [19]:
def remove_wrong_tags(label_list):
  for label in label_list:
    for idx,w in enumerate(label):
      if w in ['B-11','B-12','B-50', 'B-60', 'I-2']:
        label[idx] = 'O'

In [20]:
remove_wrong_tags(labels)

### Transformação dos dados

In [21]:
words = set()

for act in acts:
    for word in act:
        words.add(word)
#convertendo o set em uma lista
words = list(words)

words.append("ENDPAD")
words.append("UNK")

words_amt = len(words)

In [22]:
tags = set()

for label in labels:
    for tag in label:
        tags.add(tag)
tags = list(tags)
tags_amt = len(tags)

In [23]:
lab_enc = LabelEncoder()

lab_enc.fit(words)
words_i = dict(zip(lab_enc.classes_, lab_enc.transform(lab_enc.classes_)))

i_words = {}

for key in words_i:
  i_words[words_i[key]] = key

In [24]:
lab_enc = LabelEncoder()

lab_enc.fit(tags)
tags_i = dict(zip(lab_enc.classes_, lab_enc.transform(lab_enc.classes_)))

i_tags = {}

for key in tags_i:
  i_tags[tags_i[key]] = key

In [25]:
def transform_data(x,y):
    X,Y = [],[]

    for act in x:
        aux = []
        for word in act:
            aux.append(words_i[word])
        X.append(aux)

    for label in y:
        aux = []
        for word in label:
            aux.append(tags_i[word])
        Y.append(aux)

    return X,Y

In [26]:
inputs, targets = transform_data(acts,labels)

In [27]:
inputs = pad_sequences(maxlen=max_length, sequences=inputs, padding="post", value=words_i['ENDPAD'])
targets = pad_sequences(maxlen=max_length, sequences=targets, padding="post", value=tags_i["O"])

### Funções auxiliares

In [28]:
def convert_values(index_array,y_test):
  pred_tags = []
  real_tags = []

  for act in index_array:
    act_tags = []
    for w in act:
      act_tags.append(i_tags[w])
    pred_tags.append(act_tags)

  for ato in y_test:
    tags_ato = []
    for palavra in ato:
      tags_ato.append(i_tags[palavra])
    real_tags.append(tags_ato)

  return real_tags, pred_tags

# K-fold

In [29]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5, shuffle=True, random_state = 42)

### CNN-BiLSTM

In [30]:
#Aditamento Contratual
#cnnbilstm_lr = 0.001
#cnnbilstm_units = 180
#cnnbatch_size = 12
#cnnepochs = 50
'''
#Aviso de Licitação
cnnbilstm_lr = 0.0055
cnnbilstm_units = 140
cnnbatch_size = 5
cnnepochs = 50





#Suspensão
cnnbilstm_lr = 0.0065
cnnbilstm_units = 200
cnnbatch_size = 5
cnnepochs = 50 

#Revogação
cnnbilstm_lr = 0.008
cnnbilstm_units = 105
cnnbatch_size = 5
cnnepochs = 20


#Extrato de Convenio
cnnbilstm_lr = 0.003
cnnbilstm_units = 340
cnnbatch_size = 12
cnnepochs = 50'''

#Extrato de Contrato
cnnbilstm_lr = 0.003
cnnbilstm_units = 340
cnnbatch_size = 12
cnnepochs = 50

In [31]:
act_name

'EXTRATO_CONVENIO'

In [32]:
#import os, pickle
from tensorflow.keras.layers import BatchNormalization, Dropout, Conv1D, MaxPooling1D, Embedding, TimeDistributed, Bidirectional,GlobalMaxPooling1D

acc_3 = []
loss_3 = []
f1_3 = []
reports_3 = []
fold = 0

for train, test in kfold.split(inputs, targets):
 
  x_train = inputs[train]
  x_test = inputs[test]
  y_train = targets[train]
  y_test = targets[test]
          
  cnn_bilstm = Sequential()
  cnn_bilstm.add(Embedding(input_dim=words_amt, output_dim=50, input_length=max_length))
  cnn_bilstm.add(Conv1D(filters=tags_amt, kernel_size=3, padding='same', activation='relu'))
  cnn_bilstm.add(BatchNormalization())
  cnn_bilstm.add(Dropout(0.5))
  cnn_bilstm.add(Bidirectional(LSTM(cnnbilstm_units, return_sequences=True)))
  cnn_bilstm.add(TimeDistributed(Dense(tags_amt, activation="softmax")))

  adam = Adam(learning_rate=cnnbilstm_lr)

  cnn_bilstm.compile(optimizer=adam,loss="sparse_categorical_crossentropy",metrics=["accuracy"])

  early_stopping = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=5)
  callbacks = [early_stopping]

  history = cnn_bilstm.fit(x_train,y_train,batch_size=cnnbatch_size, epochs=cnnepochs)

  scores = cnn_bilstm.evaluate(x_test,y_test,verbose=0)
  acc_3.append(scores[1])
  loss_3.append(scores[0])

  predictions = cnn_bilstm.predict(x_test, verbose=0)
  #y_pred = cnn_bilstm.predict(x_test, verbose=0)
 
  predictions = np.argmax(predictions, axis=-1)
  y_pred = np.argmax(predictions, axis=-1)
  real_tags, pred_tags = convert_values(predictions,y_test)
  

  f1_3.append(f1_score(real_tags, pred_tags))
  reports_3.append(classification_report(real_tags, pred_tags))
  
  r = classification_report(real_tags,pred_tags, output_dict=True, mode='strict', scheme=IOB2)
  
  name =  act_name + '_f' + str(fold) +".npy"
  np.save('./Results/V2/CISTI/27_06/BILSTM/'+name, r)

  fold = fold + 1
  reports_3.append(r)

2023-07-11 23:12:39.292658: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/50
116/116 [==============================] - 264s 2s/step - loss: 0.3796 - accuracy: 0.9024
Epoch 2/50
116/116 [==============================] - 270s 2s/step - loss: 0.0715 - accuracy: 0.9805
Epoch 3/50
116/116 [==============================] - 273s 2s/step - loss: 0.0432 - accuracy: 0.9878
Epoch 4/50
116/116 [==============================] - 311s 3s/step - loss: 0.0311 - accuracy: 0.9912
Epoch 5/50
116/116 [==============================] - 268s 2s/step - loss: 0.0259 - accuracy: 0.9925
Epoch 6/50
116/116 [==============================] - 266s 2s/step - loss: 0.0222 - accuracy: 0.9935
Epoch 7/50
116/116 [==============================] - 276s 2s/step - loss: 0.0205 - accuracy: 0.9938
Epoch 8/50
116/116 [==============================] - 258s 2s/step - loss: 0.0180 - accuracy: 0.9945
Epoch 9/50
116/116 [==============================] - 252s 2s/step - loss: 0.0174 - accuracy: 0.9947
Epoch 10/50
116/116 [==============================] - 2091s 18s/step - loss: 0.0172 - accu

In [47]:
#pred_tags


In [70]:
f1_3[0]

0.73837784371909

In [40]:
np.mean(f1_3)

0.0

In [50]:
np.std(f1_3)

0.0

In [69]:
act_name

'AVISO_LICITACAO'

In [110]:
print(reports_3[0])

                      precision    recall  f1-score   support

        decisao_tcdf       0.00      0.00      0.00         1
modalidade_licitacao       0.75      0.63      0.69        19
    nome_responsavel       0.68      0.76      0.72        17
    numero_licitacao       0.83      0.88      0.86        17
    objeto_licitacao       0.43      0.56      0.49        18
     orgao_licitante       0.65      0.81      0.72        16
        processo_gdf       0.80      0.67      0.73        12

           micro avg       0.65      0.71      0.68       100
           macro avg       0.59      0.62      0.60       100
        weighted avg       0.68      0.71      0.69       100

